In [ ]:
import os
import yaml
import requests
import json

def _get_config(service_str):
    path = os.path.join(os.path.dirname(os.path.realpath('__file__')),'config/config.txt')
    with open(path, 'r') as config_file:
        config = yaml.load(config_file)
    return config[service_str]

#https://qpx-express-demo.itasoftware.com/

def get_flight_results(payload):
    config = _get_config('qpx_express')
    url = '{}?key={}'.format(config['url'],config['project_key'])
    r = requests.post(url, 
                      data = json.dumps(payload), 
                      headers = config['headers'])
    
    return json.loads(r.text)

def create_request(origin, destination, date):
    passengers = {}
    passengers['adultCount'] = 1
    slice = {}
    slice['origin'] = origin
    slice['destination'] = destination
    slice['date'] = date
    slice['maxStops'] = 0
    request = {}
    request['passengers'] = passengers
    request['slice'] = [slice]
    request['solutions'] = 500
    return {'request': request}

In [ ]:
results = get_flight_results(create_request('BOS','LAX','2016-06-01'))

In [ ]:
reference_data = results['trips']['data']
trip_options = results['trips']['tripOption']

In [ ]:
for trip_option in trip_options:
    code = trip_option['slice'][0]['segment'][0]['flight']['carrier']
    carrier = [carrier['name'] for carrier in reference_data['carrier'] if carrier['code']==code][0]
    number = trip_option['slice'][0]['segment'][0]['flight']['number']
    price = trip_option['saleTotal']
    arrival = trip_option['slice'][0]['segment'][0]['leg'][0]['arrivalTime']
    departure = trip_option['slice'][0]['segment'][0]['leg'][0]['departureTime']
    print "{} {} {} {} {}".format(carrier, number, price, arrival, departure)